# Agent0.1

In [ ]:
!pip install -q \
langchain \
langchain-community \
langchain-groq \
langchain-text-splitters \
chromadb \
duckduckgo-search \
sentence-transformers \
groq \
pyngrok \
streamlit


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [3]:
import os
os.environ["GROQ_API_KEY"] ="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"

In [4]:
import os
os.makedirs("data", exist_ok=True)
os.makedirs("db", exist_ok=True)


In [5]:
with open("data/notes.txt", "w") as f:
    f.write("""
Artificial Intelligence is the simulation of human intelligence by machines.
It includes machine learning, deep learning, and natural language processing.
""")


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import os

def load_docs():
    docs = []
    for file in os.listdir("data"):
        if file.endswith(".txt"):
            loader = TextLoader(f"data/{file}")
            docs.extend(loader.load())
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(docs)

def create_db():
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma.from_documents(load_docs(), embeddings, persist_directory="db")
    db.persist()

create_db()


In [7]:
from langchain_groq import ChatGroq
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from duckduckgo_search import DDGS

llm = ChatGroq(model="llama-3.3-70b-versatile")

def web_search(q):
    return "\n".join([r["body"] for r in DDGS().text(q, max_results=3)])

db = Chroma(
    persist_directory="db",
    embedding_function=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
)

def doc_search(q):
    results = db.similarity_search(q, k=3)
    return "\n".join([r.page_content for r in results])

def ask(q):
    prompt = f"""
You are a helpful AI study assistant.

Web info:
{web_search(q)}

User notes:
{doc_search(q)}

Answer clearly:
{q}
"""
    return llm.invoke(prompt).content


/tmp/ipython-input-2845828064.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(


In [8]:
def create_plan(goal):
    return llm.invoke(
        f"Create a simple, realistic 30-day study plan for: {goal}"
    ).content

def reflect(topic):
    return llm.invoke(
        f"Ask 5 deep questions and 3 new angles about: {topic}"
    ).content

def challenge(idea):
    return llm.invoke(
        f"Critically analyze this idea: {idea}"
    ).content


In [ ]:
print(ask("Search best free Python course"))
print(ask("What is in my notes about AI?"))
print(create_plan("Learn Python"))
print(reflect("AI in education"))


In [ ]:
%%writefile app.py
import streamlit as st
from __main__ import ask, create_plan, reflect, challenge

st.set_page_config(page_title="AI Study Agent")
st.title("🧠 AI Study & Research Agent")

tab1, tab2, tab3 = st.tabs(["Chat", "Study Plan", "Thinking"])

with tab1:
    q = st.text_input("Ask me anything")
    if st.button("Ask"):
        st.write(ask(q))

with tab2:
    goal = st.text_input("Your study goal")
    if st.button("Create Plan"):
        st.write(create_plan(goal))

with tab3:
    text = st.text_area("Topic or idea")
    if st.button("Reflect"):
        st.write(reflect(text))
    if st.button("Challenge"):
        st.write(challenge(text))


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("1g3JDCIN1SkvhSQk767MrGLc4hw_6WLLB6dHC9UX7JRevdUbr")


In [ ]:
from pyngrok import ngrok
import subprocess

process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

public_url = ngrok.connect(8501)
print("🌍 Your AI Agent is live at:", public_url)


In [ ]:
%%writefile backend.py
from langchain_groq import ChatGroq
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from duckduckgo_search import DDGS

llm = ChatGroq(model="llama-3.3-70b-versatile")

def web_search(q):
    return "\n".join([r["body"] for r in DDGS().text(q, max_results=3)])

db = Chroma(
    persist_directory="db",
    embedding_function=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
)

def doc_search(q):
    results = db.similarity_search(q, k=3)
    return "\n".join([r.page_content for r in results])

def ask(q):
    prompt = f"""
You are a helpful AI study assistant.

Web info:
{web_search(q)}

User notes:
{doc_search(q)}

Answer clearly:
{q}
"""
    return llm.invoke(prompt).content

def create_plan(goal):
    return llm.invoke(
        f"Create a simple, realistic 30-day study plan for: {goal}"
    ).content

def reflect(topic):
    return llm.invoke(
        f"Ask 5 deep questions and 3 new angles about: {topic}"
    ).content

def challenge(idea):
    return llm.invoke(
        f"Critically analyze this idea: {idea}"
    ).content


In [ ]:
%%writefile app.py
import streamlit as st
from backend import ask, create_plan, reflect, challenge

st.set_page_config(page_title="AI Study Agent")
st.title("🧠 AI Study & Research Agent")

tab1, tab2, tab3 = st.tabs(["Chat", "Study Plan", "Thinking"])

with tab1:
    q = st.text_input("Ask me anything")
    if st.button("Ask"):
        st.write(ask(q))

with tab2:
    goal = st.text_input("Your study goal")
    if st.button("Create Plan"):
        st.write(create_plan(goal))

with tab3:
    text = st.text_area("Topic or idea")
    if st.button("Reflect"):
        st.write(reflect(text))
    if st.button("Challenge"):
        st.write(challenge(text))


In [ ]:
from pyngrok import ngrok
import subprocess

process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

public_url = ngrok.connect(8501)
print("🌍 Your AI Agent is live at:", public_url)
